In [ ]:
#مرحله ی اول
#بر اساس نام حرکت ورزشی
import os
import cv2
import numpy as np
import mediapipe as mp
from ultralytics import YOLO
import csv

# بارگذاری مدل YOLO (نسخه سبک)
model = YOLO("yolov8n.pt")

# تنظیم MediaPipe Pose برای تشخیص اسکلت بدن
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1)

# تنظیمات Optical Flow (Lucas-Kanade)
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# مسیر ذخیره‌سازی نقاط کلیدی
output_dir = "keypoints_output_name_1"
os.makedirs(output_dir, exist_ok=True)

def save_keypoints_to_csv(keypoints, exercise_name, frame_number):
    """ ذخیره نقاط کلیدی در فایل CSV با نام حرکت ورزشی """
    output_file = os.path.join(output_dir, f"{exercise_name}_frame_{frame_number}.csv")
    with open(output_file, mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['x', 'y'])  # عنوان ستون‌ها
        for pt in keypoints:
            writer.writerow(pt)  # ذخیره نقاط کلیدی

def process_video(video_path, exercise_name):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    if not ret:
        print(f"❌ خطا در خواندن ویدیو: {video_path}")
        return

    # کاهش رزولوشن اولیه به 320x240
    prev_frame = cv2.resize(prev_frame, (320, 240))
    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_keypoints = []  # نقاط کلیدی قبلی (به عنوان لیست از (x, y))
    frame_count = 0

    yolo_interval = 10  # هر 10 فریم، تشخیص YOLO/MediaPipe انجام شود

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # کاهش رزولوشن فریم به 320x240
        frame = cv2.resize(frame, (320, 240))
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_count += 1

        if frame_count % yolo_interval == 5:
            # اجرای YOLO برای تشخیص ورزشکار (فقط انسان)
            results = model(frame)
            new_keypoints = []
            for result in results:
                for box in result.boxes:
                    if int(box.cls) == 0:  # کلاس 0 برای انسان
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        x1 = max(0, x1); y1 = max(0, y1)
                        x2 = min(frame.shape[1], x2); y2 = min(frame.shape[0], y2)
                        roi = frame[y1:y2, x1:x2]

                        # اجرای MediaPipe برای استخراج نقاط بدن از ROI
                        rgb_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                        results_pose = pose.process(rgb_roi)
                        if results_pose.pose_landmarks:
                            for lm in results_pose.pose_landmarks.landmark:
                                x = int(x1 + lm.x * (x2 - x1))
                                y = int(y1 + lm.y * (y2 - y1))
                                new_keypoints.append((x, y))
            if new_keypoints:
                prev_keypoints = new_keypoints
                save_keypoints_to_csv(new_keypoints, exercise_name, frame_count)
        else:
            # به‌روزرسانی نقاط کلیدی با Optical Flow در فریم‌های بین تشخیص
            if prev_keypoints:
                p0 = np.array(prev_keypoints, dtype=np.float32).reshape(-1, 1, 2)
                p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, p0, None, **lk_params)
                if p1 is not None:
                    updated_points = []
                    for i, (new, status) in enumerate(zip(p1, st)):
                        if status[0] == 1:
                            x, y = new.ravel()
                            updated_points.append((int(x), int(y)))
                    if updated_points:
                        prev_keypoints = updated_points

        # رسم نقاط کلیدی روی فریم
        for pt in prev_keypoints:
            cv2.circle(frame, pt, 3, (0, 255, 0), -1)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        prev_gray = gray.copy()

    cap.release()
    cv2.destroyAllWindows()

# جمع‌آوری مسیر ویدیوها از پوشه
video_dir = r"D:\aiocup\challesh_5\New folder (3)\badansazi2"
video_paths = []
for folder_name in os.listdir(video_dir):
    folder_path = os.path.join(video_dir, folder_name)
    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if file_path.endswith(('.mp4', '.avi')):
                exercise_name = folder_name  # نام حرکت ورزشی از پوشه گرفته می‌شود
                video_paths.append((file_path, exercise_name))

# پردازش ویدیوها
for vp, exercise in video_paths:
    process_video(vp, exercise)


In [ ]:
#مرحله دوم
import os
import pandas as pd
import numpy as np
import collections

# مسیر پوشه حاوی فایل‌های CSV استخراج‌شده
csv_dir = "keypoints_output_name_1"

# پوشه برای ذخیره فایل‌های اصلاح‌شده
fixed_dir = "fixed_keypoints_output"
os.makedirs(fixed_dir, exist_ok=True)

# دیکشنری برای گروه‌بندی فایل‌ها به تفکیک حرکت ورزشی
exercise_files = {}
for file in os.listdir(csv_dir):
    if file.endswith(".csv"):
        exercise_name = file.split("_frame_")[0]
        exercise_files.setdefault(exercise_name, []).append(file)

# تعیین تعداد استاندارد نقاط کلیدی برای هر حرکت با استفاده از حالت (mode)
expected_counts = {}
for exercise, files in exercise_files.items():
    counts = []
    for f in files:
        file_path = os.path.join(csv_dir, f)
        df = pd.read_csv(file_path)
        counts.append(df.shape[0])  # تعداد ردیف‌ها (هر ردیف یک نقطه کلیدی)
    # تعیین تعداد رایج نقاط کلیدی
    count_mode = collections.Counter(counts).most_common(1)[0][0]
    expected_counts[exercise] = count_mode
    print(f"For exercise '{exercise}', expected keypoints: {count_mode}")

# اصلاح فایل‌ها بر اساس تعداد استاندارد و ذخیره آن‌ها در پوشه جدید
for exercise, files in exercise_files.items():
    expected = expected_counts[exercise]
    for f in files:
        file_path = os.path.join(csv_dir, f)
        df = pd.read_csv(file_path)
        current_count = df.shape[0]
        fixed_df = df.copy()
        
        if current_count > expected:
            # برش (crop) ردیف‌های اضافی
            fixed_df = fixed_df.iloc[:expected, :]
            print(f"File '{f}': cropped from {current_count} to {expected} keypoints.")
        elif current_count < expected:
            # اضافه کردن ردیف‌های خالی (NaN) برای تکمیل
            pad_count = expected - current_count
            pad_df = pd.DataFrame(np.nan, index=range(pad_count), columns=df.columns)
            fixed_df = pd.concat([fixed_df, pad_df], ignore_index=True)
            print(f"File '{f}': padded with {pad_count} missing keypoints.")
        
        # ذخیره فایل اصلاح‌شده در پوشه جدید با همان نام فایل
        output_path = os.path.join(fixed_dir, f)
        fixed_df.to_csv(output_path, index=False)

print("✅ اصلاح فایل‌ها و ذخیره در پوشه جدید به پایان رسید!")


For exercise 'اسکوات (Squats)', expected keypoints: 33
For exercise 'جلو بازو (Bicep Curls)', expected keypoints: 33
For exercise 'ددلیفت (Deadlifts)', expected keypoints: 33
For exercise 'فلای سینه (Chest Fly)', expected keypoints: 33
For exercise 'لت (Lat Pulldown)', expected keypoints: 33
For exercise 'لنج (Lunges)', expected keypoints: 33
For exercise 'پرس سینه (Bench Press)', expected keypoints: 33
For exercise 'پرس شانه (Overhead Shoulder Press)', expected keypoints: 33
For exercise 'پرس پا (Leg Press)', expected keypoints: 33
For exercise 'پشت بازو (Tricep Dips)', expected keypoints: 33
For exercise 'پلانک (Plank)', expected keypoints: 33
For exercise 'پوش آپ (Push-ups)', expected keypoints: 33
For exercise 'پول آپ (Pull-ups)', expected keypoints: 33
For exercise 'کراس اور (Cable Crossover)', expected keypoints: 33
For exercise 'کرانچ (Crunches)', expected keypoints: 33
File 'جلو بازو (Bicep Curls)_frame_105.csv': cropped from 66 to 33 keypoints.
File 'جلو بازو (Bicep Curls)_fra

In [ ]:
#مرحله سوم جداسازی
import os
import pandas as pd
import numpy as np

# مسیر پوشه حاوی فایل‌های CSV
csv_dir = "fixed_keypoints_output"

# بررسی وجود پوشه
if not os.path.exists(csv_dir):
    raise FileNotFoundError(f"❌ مسیر '{csv_dir}' یافت نشد!")

# دیکشنری برای ذخیره داده‌ها به تفکیک حرکت ورزشی
# کلید: نام حرکت ورزشی، مقدار: لیستی از بردارهای ویژگی استخراج‌شده
dataset_dict = {}

# لیست تمام فایل‌های CSV در پوشه
csv_files = [f for f in os.listdir(csv_dir) if f.endswith(".csv")]
if not csv_files:
    raise ValueError("❌ هیچ فایل CSV‌ای در مسیر مشخص‌شده یافت نشد!")

# دیکشنری برای ذخیره تعداد نقاط کلیدی مورد انتظار برای هر حرکت (برای یکپارچگی داده‌ها)
expected_num_keypoints = {}

# پردازش فایل‌های CSV
for file in csv_files:
    file_path = os.path.join(csv_dir, file)
    
    try:
        # استخراج نام حرکت ورزشی از نام فایل (قبل از "_frame_")
        exercise_name = file.split("_frame_")[0]

        # خواندن داده‌های CSV
        df = pd.read_csv(file_path)
        
        # بررسی ساختار صحیح داده‌ها
        if df.shape[1] != 2 or list(df.columns) != ["x", "y"]:
            print(f"⚠️ فایل '{file}' دارای ساختار نامعتبر است و نادیده گرفته می‌شود!")
            continue
        
        # تبدیل نقاط (x, y) به یک بردار ویژگی (آرایه یک‌بعدی)
        keypoints = df.to_numpy().flatten()
        
        # بررسی یکپارچگی تعداد نقاط کلیدی برای هر حرکت
        if exercise_name not in expected_num_keypoints:
            expected_num_keypoints[exercise_name] = len(keypoints)
        elif len(keypoints) != expected_num_keypoints[exercise_name]:
            print(f"⚠️ فایل '{file}' دارای تعداد نقاط کلیدی متفاوت برای حرکت '{exercise_name}' است و نادیده گرفته می‌شود!")
            continue
        
        # اضافه کردن بردار ویژگی به دیکشنری
        if exercise_name not in dataset_dict:
            dataset_dict[exercise_name] = []
        dataset_dict[exercise_name].append(keypoints)
        
    except Exception as e:
        print(f"⚠️ خطا در پردازش فایل '{file}': {e}")

# بررسی اینکه پس از فیلتر کردن، حداقل یک داده وجود داشته باشد
if not dataset_dict:
    raise ValueError("❌ پس از فیلتر کردن داده‌ها، هیچ فایل معتبری باقی نماند!")

# ایجاد پوشه برای ذخیره دیتاست‌های نهایی به تفکیک حرکت ورزشی
output_dir = "final_datasets_by_exercise_1"
os.makedirs(output_dir, exist_ok=True)

# ایجاد دیتافریم و ذخیره هر دیتاست به تفکیک حرکت ورزشی
for exercise, features_list in dataset_dict.items():
    if not features_list:
        continue
    # محاسبه تعداد نقاط کلیدی و تقسیم آن به x و y
    num_keypoints = expected_num_keypoints[exercise]
    num_points = num_keypoints // 2
    
    # تعریف نام ستون‌ها برای دیتافریم
    col_names = [f"x{i}" for i in range(num_points)] + [f"y{i}" for i in range(num_points)]
    
    # ساخت دیتافریم برای حرکت جاری
    df_ex = pd.DataFrame(features_list, columns=col_names)
    
    # مسیر فایل خروجی برای حرکت جاری
    output_file = os.path.join(output_dir, f"{exercise}.csv")
    
    # ذخیره دیتافریم در فایل CSV
    df_ex.to_csv(output_file, index=False)
    print(f"✅ دیتاست برای حرکت '{exercise}' در فایل '{output_file}' ذخیره شد!")


✅ دیتاست برای حرکت 'اسکوات (Squats)' در فایل 'final_datasets_by_exercise_1\اسکوات (Squats).csv' ذخیره شد!
✅ دیتاست برای حرکت 'جلو بازو (Bicep Curls)' در فایل 'final_datasets_by_exercise_1\جلو بازو (Bicep Curls).csv' ذخیره شد!
✅ دیتاست برای حرکت 'ددلیفت (Deadlifts)' در فایل 'final_datasets_by_exercise_1\ددلیفت (Deadlifts).csv' ذخیره شد!
✅ دیتاست برای حرکت 'فلای سینه (Chest Fly)' در فایل 'final_datasets_by_exercise_1\فلای سینه (Chest Fly).csv' ذخیره شد!
✅ دیتاست برای حرکت 'لت (Lat Pulldown)' در فایل 'final_datasets_by_exercise_1\لت (Lat Pulldown).csv' ذخیره شد!
✅ دیتاست برای حرکت 'لنج (Lunges)' در فایل 'final_datasets_by_exercise_1\لنج (Lunges).csv' ذخیره شد!
✅ دیتاست برای حرکت 'پرس سینه (Bench Press)' در فایل 'final_datasets_by_exercise_1\پرس سینه (Bench Press).csv' ذخیره شد!
✅ دیتاست برای حرکت 'پرس شانه (Overhead Shoulder Press)' در فایل 'final_datasets_by_exercise_1\پرس شانه (Overhead Shoulder Press).csv' ذخیره شد!
✅ دیتاست برای حرکت 'پرس پا (Leg Press)' در فایل 'final_datasets_by_exe

In [ ]:
#مرحله چهارم
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import joblib

# ----------------------------- #
# مرحله ۱: بارگذاری دیتاست‌ها
# ----------------------------- #

# مسیر پوشه دیتاست‌های نهایی به تفکیک حرکت ورزشی
data_dir = "final_datasets_by_exercise_1"

X_data = []
y_data = []

# هر فایل CSV یک حرکت ورزشی مشخص را نشان می‌دهد.
for file in os.listdir(data_dir):
    if file.endswith(".csv"):
        # نام حرکت ورزشی از نام فایل استخراج می‌شود (بدون پسوند)
        label = file.replace(".csv", "")
        df = pd.read_csv(os.path.join(data_dir, file))
        # فرض می‌شود که هر ردیف یک نمونه (یک فریم) از نقاط کلیدی است
        X_data.extend(df.values)
        y_data.extend([label] * len(df))

X_data = np.array(X_data)
y_data = np.array(y_data)

print("✅ شکل اولیه داده‌ها (X):", X_data.shape)
print("✅ برچسب‌های یکتا:", np.unique(y_data))

# ----------------------------- #
# مرحله ۲: پیش‌پردازش داده‌ها
# ----------------------------- #

# نرمال‌سازی داده‌ها
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_data)

# ذخیره scaler برای استفاده‌های بعدی
joblib.dump(scaler, "lstm_scaler.pkl")
print("✅ Scaler ذخیره شد: lstm_scaler.pkl")

# رمزگذاری برچسب‌ها
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_data)
y_categorical = to_categorical(y_encoded)

# ذخیره label encoder
joblib.dump(label_encoder, "label_encoder.pkl")
print("✅ LabelEncoder ذخیره شد: label_encoder.pkl")

# تقسیم داده به آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_categorical, test_size=0.2, random_state=42, stratify=y_categorical
)
print(f"✅ تقسیم داده: آموزش = {X_train.shape[0]} نمونه, تست = {X_test.shape[0]} نمونه")

# تغییر شکل داده‌ها به صورت مناسب برای CNN-LSTM:
# فرض: هر نمونه به صورت یک فریم است؛ بنابراین توالی طول 1 داریم
# اگر در آینده توالی‌های چند فریمی دارید، باید این قسمت تغییر کند.
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test  = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print("✅ شکل داده‌های آموزش:", X_train.shape)
print("✅ شکل داده‌های تست:", X_test.shape)

# ----------------------------- #
# مرحله ۳: ساخت مدل CNN-LSTM
# ----------------------------- #

num_features = X_train.shape[1]  # تعداد ویژگی‌ها (مثلاً 66 اگر 33 نقطه با ۲ مختصات)
num_classes = y_categorical.shape[1]

model = Sequential()
# لایه‌های CNN برای استخراج ویژگی‌های محلی
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(num_features, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))
# لایه LSTM برای مدل‌سازی وابستگی‌های زمانی (در اینجا طول توالی برابر 1 است؛ برای توالی‌های طولانی‌تر کاربرد دارد)
model.add(LSTM(64, return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ----------------------------- #
# مرحله ۴: آموزش مدل
# ----------------------------- #

history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))

# ----------------------------- #
# مرحله ۵: ذخیره مدل آموزش دیده
# ----------------------------- #

model.save("lstm_model.h5")
print("✅ مدل ذخیره شد: lstm_model.h5")


✅ شکل اولیه داده‌ها (X): (1261, 66)
✅ برچسب‌های یکتا: ['اسکوات (Squats)' 'جلو بازو (Bicep Curls)' 'ددلیفت (Deadlifts)' 'فلای سینه (Chest Fly)' 'لت (Lat Pulldown)' 'پرس سینه (Bench Press)' 'پرس پا (Leg Press)' 'پوش آپ (Push-ups)' 'کراس اور (Cable Crossover)' 'کرانچ (Crunches)']
✅ Scaler ذخیره شد: lstm_scaler.pkl
✅ LabelEncoder ذخیره شد: label_encoder.pkl
✅ تقسیم داده: آموزش = 1008 نمونه, تست = 253 نمونه
✅ شکل داده‌های آموزش: (1008, 66, 1)
✅ شکل داده‌های تست: (253, 66, 1)
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 64, 64)            256       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 32, 64)           0         
 1D)                                                             
                                                                 
 dropout_10 (Dropout)        (None, 32, 6

In [ ]:
#مرحله پنجم
import cv2
import numpy as np
import mediapipe as mp
import joblib
from tensorflow.keras.models import load_model

# مسیرهای ذخیره مدل‌ها و scaler و label encoder
MODEL_PATH = "lstm_model.h5"
SCALER_PATH = "lstm_scaler.pkl"
LABEL_ENCODER_PATH = "label_encoder.pkl"

# بارگذاری مدل، scaler و label encoder
model = load_model(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)
label_encoder = joblib.load(LABEL_ENCODER_PATH)

# تنظیمات MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1)
# تعداد نقاط کلیدی: فرض بر این است که 33 نقطه با 2 مختصات (x,y) استخراج می‌شود
NUM_FEATURES = 33 * 2

def extract_keypoints(frame):
    """
    از یک فریم، نقاط کلیدی (x,y) بدن را استخراج می‌کند.
    در صورت عدم شناسایی، یک آرایه از صفرها برمی‌گرداند.
    """
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    keypoints = []
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            keypoints.append(landmark.x)
            keypoints.append(landmark.y)
    else:
        keypoints = [0.0] * NUM_FEATURES
    return np.array(keypoints)

def predict_exercise_from_video(video_path):
    # باز کردن ویدیو
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ خطا در باز کردن ویدیو: {video_path}")
        return None

    # تعیین تعداد فریم‌ها و انتخاب فریم میانی به عنوان نمونه نماینده
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    mid_frame_no = frame_count // 2
    cap.set(cv2.CAP_PROP_POS_FRAMES, mid_frame_no)
    ret, frame = cap.read()
    cap.release()

    if not ret:
        print("❌ خطا در خواندن فریم از ویدیو.")
        return None

    # تغییر اندازه فریم (در صورت نیاز، همان اندازه‌ای که در آموزش استفاده شده، مثلاً 320x240)
    frame = cv2.resize(frame, (320, 240))

    # استخراج نقاط کلیدی
    keypoints = extract_keypoints(frame)
    if keypoints.shape[0] != NUM_FEATURES:
        print("❌ تعداد نقاط کلیدی استخراج‌شده نادرست است.")
        return None

    # تغییر شکل به صورت (1, تعداد ویژگی‌ها, 1) چون مدل ما انتظار ورودی با شکل (batch, features, 1) دارد
    input_data = keypoints.reshape(1, NUM_FEATURES)
    # نرمال‌سازی داده‌ها با scaler ذخیره‌شده
    input_scaled = scaler.transform(input_data)
    # تغییر شکل به سه بعدی برای مدل (1, NUM_FEATURES, 1)
    input_final = input_scaled.reshape(1, NUM_FEATURES, 1)

    print("🔍 Keypoints shape:", keypoints.shape)
    print("📌 Keypoints sample:", keypoints[:10])  # ده تا اولی رو چاپ کن
    
    
    
    # پیش‌بینی با مدل
    prediction_probs = model.predict(input_final)
    predicted_index = np.argmax(prediction_probs, axis=1)[0]
    predicted_label = label_encoder.inverse_transform([predicted_index])[0]
    return predicted_label

# مثال استفاده:
video_file = "ppp.mp4"  # مسیر ویدیوی مورد نظر خود را وارد کنید
predicted_exercise = predict_exercise_from_video(video_file)
if predicted_exercise is not None:
    print("نوع حرکت ورزشی تشخیص داده شده:", predicted_exercise)


🔍 Keypoints shape: (66,)
📌 Keypoints sample: [    0.32086     0.67493     0.31178     0.66364     0.31195     0.66177     0.31217     0.65976     0.31168     0.66354]
1/1 [==============================] - 1s 606ms/step
نوع حرکت ورزشی تشخیص داده شده: پوش آپ (Push-ups)


In [62]:
print("📊 کلاس‌هایی که مدل می‌شناسه:", label_encoder.classes_)


📊 کلاس‌هایی که مدل می‌شناسه: ['اسکوات (Squats)' 'جلو بازو (Bicep Curls)' 'ددلیفت (Deadlifts)' 'فلای سینه (Chest Fly)' 'لت (Lat Pulldown)' 'پرس سینه (Bench Press)' 'پرس پا (Leg Press)' 'پوش آپ (Push-ups)' 'کراس اور (Cable Crossover)' 'کرانچ (Crunches)']


In [21]:
#مرحله ششم
import cv2
import numpy as np
import mediapipe as mp
import joblib
from tensorflow.keras.models import load_model

# مسیرهای ذخیره مدل‌ها و scaler و label encoder
MODEL_PATH = "lstm_model.h5"
SCALER_PATH = "lstm_scaler.pkl"
LABEL_ENCODER_PATH = "label_encoder.pkl"

# بارگذاری مدل، scaler و label encoder
model = load_model(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)
label_encoder = joblib.load(LABEL_ENCODER_PATH)

# تنظیمات MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1)
mp_drawing = mp.solutions.drawing_utils
# تعداد نقاط کلیدی: فرض بر این است که 33 نقطه با 2 مختصات (x,y) استخراج می‌شود
NUM_FEATURES = 33 * 2

def extract_keypoints(frame):
    """
    از یک فریم، نقاط کلیدی (x,y) بدن را استخراج می‌کند.
    در صورت عدم شناسایی، یک آرایه از صفرها برمی‌گرداند.
    """
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    keypoints = []
    if results.pose_landmarks:
        for landmark in results.pose_landmarks.landmark:
            keypoints.append(landmark.x)
            keypoints.append(landmark.y)
    else:
        keypoints = [0.0] * NUM_FEATURES
    return np.array(keypoints), results.pose_landmarks

def predict_exercise_from_video(video_path=None):
    cap = cv2.VideoCapture(video_path if video_path else 0)  # اگر ویدیو موجود نباشد، از وب‌کم استفاده کن
    if not cap.isOpened():
        print("❌ خطا در باز کردن ویدیو.")
        return None

    # تنظیم اندازه ویدیو
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # تغییر اندازه فریم (در صورت نیاز، همان اندازه‌ای که در آموزش استفاده شده، مثلاً 320x240)
        frame_resized = cv2.resize(frame, (320, 240))

        # استخراج نقاط کلیدی
        keypoints, landmarks = extract_keypoints(frame_resized)
        if keypoints.shape[0] != NUM_FEATURES:
            print("❌ تعداد نقاط کلیدی استخراج‌شده نادرست است.")
            continue

        # نرمال‌سازی داده‌ها با scaler ذخیره‌شده
        input_scaled = scaler.transform(keypoints.reshape(1, -1))

        # تغییر شکل به سه بعدی برای مدل (1, NUM_FEATURES, 1)
        input_final = input_scaled.reshape(1, NUM_FEATURES, 1)

        # پیش‌بینی با مدل
        prediction_probs = model.predict(input_final)
        predicted_index = np.argmax(prediction_probs, axis=1)[0]
        predicted_label = label_encoder.inverse_transform([predicted_index])[0]

        # نمایش نوع حرکت ورزشی تشخیص داده‌شده
        cv2.putText(frame, f"Predicted: {predicted_label}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # رسم نقاط کلیدی و خطوط بین نقاط
        if landmarks:
            for i in range(33):  # رسم نقاط کلیدی
                x = int(landmarks.landmark[i].x * frame_width)
                y = int(landmarks.landmark[i].y * frame_height)
                cv2.circle(frame, (x, y), 5, (0, 255, 255), -1)  # نقاط کلیدی به رنگ زرد

            # رسم خطوط بین نقاط کلیدی (وصل کردن نقاط مختلف بدن)
            connections = mp_pose.POSE_CONNECTIONS
            for connection in connections:
                start_idx, end_idx = connection
                start = landmarks.landmark[start_idx]
                end = landmarks.landmark[end_idx]
                start_coords = int(start.x * frame_width), int(start.y * frame_height)
                end_coords = int(end.x * frame_width), int(end.y * frame_height)
                cv2.line(frame, start_coords, end_coords, (0, 255,0), 2)  # خطوط به رنگ قرمز

        # نمایش ویدیو
        cv2.imshow("Exercise Feedback", frame)

        # زمانی که کاربر کلید 'q' را بزند، از ویدیو خارج می‌شود
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# استفاده از تابع برای ویدیو زنده یا ویدیوی بارگذاری شده
video_file = "ppp.mp4"  # برای استفاده از ویدیو، این مسیر را مشخص کنید
predict_exercise_from_video(video_file)  # برای استفاده از وب‌کم، ویدیو را خالی بگذارید یا None قرار دهید


1/1 [==============================] - 0s 32ms/step


KeyboardInterrupt: 